In [ ]:
#setting up spotipy
import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

# if len(sys.argv) > 1:
#     username = sys.argv[1]
# else:
#     print ("Usage: %s username" % (sys.argv[0],))
#     sys.exit()
    
# token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
#                                    client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='ff9ff248e80e428bbb9796e1a7d62aeb', client_secret='7f70a7c884494c6c96d2e6f03c4d2388')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
import pandas as pd
import numpy as np

# sample = pd.read_csv('sample.csv')

In [ ]:
df = pd.read_csv('billboard_weekly_id.csv')

In [ ]:
df.shape

In [ ]:
df[df['sp_id'].notnull()].to_csv('billboard_with_ids.csv')

In [ ]:
df[df['sp_id'].isnull()]

In [ ]:
feature_names = ['duration_ms', 'key', 'mode', 'time_signature', 'acousticness', 'danceability', 
                'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']

In [ ]:
def get_features_with_id(id):
    track = sp.audio_features(id)[0]
    
    return [track[feature] for feature in feature_names]
    

In [ ]:
column_names = list(sample.columns)
column_names.extend(feature_names)
sample = sample.reindex(columns = column_names)

In [ ]:
sample[feature_names] = sample['Spotify_Track_ID'].apply(lambda x: pd.Series(get_features_with_id(x)))

In [ ]:
column_names = list(sample.columns)
column_names.extend(feature_names)
sample.reindex(columns = column_names)

In [ ]:
analysis = sp.audio_analysis('2XkuSbp5say8nZW8g6156Z')

In [ ]:
billboard_weekly = pd.read_csv('https://query.data.world/s/fmip5pplwfadgvsbinribf5sgs7rcx')

In [ ]:
def get_track_id(track_name = "", artist = ""):
    # 'featuring' problem
    if 'Featuring' in artist:
        artist = artist.split('Featuring')[0]

#     print(track_name, artist)
    
    q = track_name + ' ' + artist
    search_results = sp.search(q)
    
    #Index Error
    try:
        track_id = search_results['tracks']['items'][0]['id']
    except IndexError:
        track_id = 'null'
            
    return track_id

In [ ]:
def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)

def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    print(dfm.shape[0])
    return np.split(dfm, indices)

chunks = split(billboard_weekly, 5000)

In [ ]:
count = 0
for c in chunks:
    count += 1
    print('Chunk #: ' + str(count))
    token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
                                   client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')
    
    sp = spotipy.Spotify(auth=token)
    c = parallelize_dataframe(c, get_sp_id, 4)

In [ ]:
billboard_weekly.head()

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

def get_sp_id(row):
    artist = row.Performer
    track_name = row.Song
    
    if 'Featuring' in row.Performer:
        artist = artist.split('Featuring')[0]
    
    if '&' in artist:
        artist = artist.split('&')[0]
        
    if 'With' in artist:
        artist = artist.split('With')[0]
        
#     print(track_name, artist)
    q = track_name + ' ' + artist
    search_results = sp.search(q)
    
    #Index Error
    try:
        track_id = search_results['tracks']['items'][0]['id']
    except IndexError:
        track_id = 'null'
            
    return track_id



In [ ]:
#test:
%time result = chunks[0].parallel_apply(get_sp_id, axis=1)


In [ ]:
len(result)

In [ ]:
count = 0
result = []
for chunk in chunks:
    count += 1
    print('Chunk #: ' + str(count))
#     sp = spotipy.Spotify(auth=token)
    
    %time chunk_result = chunk.parallel_apply(get_sp_id, axis=1)
    
    result.extend(chunk_result)


In [ ]:
%time id_list = billboard_weekly.parallel_apply(get_sp_id, axis=1)

In [ ]:
len(id_list)

In [ ]:
billboard_weekly.shape

In [ ]:
billboard_weekly['sp_id'] = id_list

In [ ]:
billboard_weekly.to_csv('billboard_weekly_id.csv')

In [ ]:
sum(id_list == 'null')

In [ ]:
null_df = billboard_weekly[billboard_weekly['sp_id'] == 'null']

In [ ]:
sp.search('Return To Sender')

In [ ]:
%time null_id_list = null_df.parallel_apply(get_sp_id, axis=1)

In [ ]:
sum(null_id_list=='null')

In [ ]:
null_id_list.head()

In [ ]:
null_df[null_df['sp_id'] != 'null']

In [ ]:
null_df['sp_id'] = null_id_list

In [ ]:
df = pd.merge(billboard_weekly, null_df, how='left', on='sp_id')

In [ ]:
null_id_list